# Cine21 Crawling

In [15]:
import time
from multiprocessing import Pool
# parmap 알아보기 이 함수가 얼마나 더 도는지 알려줌
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [16]:
# StaleElementReferenceException means the element is no longer in the DOM, or it changed.
# The following code will help you find the element by controlling and ignoring StaleElementExceptions and
# handling them just like any other NoSuchElementException.
# It waits for the element to NOT be stale, just like it waits for the element to be present.
# It also serves as a good example on how to properly wait for conditions in Selenium.

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

# my_element_id = 'something123'
# ignored_exceptions = (NoSuchElementException, StaleElementReferenceException)
# your_element = WebDriverWait(driver, 10, ignored_exceptions = ignored_exceptions)\
#                         .until(expected_conditions.presence_of_element_located((By.ID, my_element_id)))

In [17]:
from selenium import webdriver

driver = webdriver.Chrome("./chromedriver")

# 씨네21 영화 검색 페이지 접속 (''검색 결과)
driver.get("http://www.cine21.com/")

In [5]:
#영화제목으로 검색
### ! 영화제목 리스트 돌면서 차례대로
result = []
# example_titles = ['007 노 타임 투 다이', '보이스', '기적', '용과 주근깨 공주', '극장판 포켓몬스터: 기적의 아이, 코코',
#                   '샹치와 텐 링즈의 전설', '극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들',
#                   '최선의 삶', '코다', '아임 유어 맨']
data = {'title':['기적', '극장판 포켓몬스터: 정글의 아이, 코코'],
        'release_date':['2021.09.15', '2021.09.15'],
        'actor':['박정민,이성민,윤아,이수경,김강훈', '마츠모토 리카,오오타니 이쿠에,하야시바라 메구미,미키 신이치로,이누야마 이누코'],
            'director':['이장훈', '야지마 테츠오'],
            'summary': ["오갈 수 있는 길은 기찻길밖에 없지만 정작 기차역은 없는 마을. 오늘부로 청와대에 딱 54번째 편지를 보낸 ‘준경’(박정민)의 목표는 단 하나! 바로 마을에 기차역이 생기는 것이다. 기차역은 어림없다는 원칙주의 기관사 아버지 ‘태윤’(이성민)의 반대에도 누나 ‘보경’(이수경)과 마을에 남는 걸 고집하며 왕복 5시간 통학길을 오가는 ‘준경’. 그의 엉뚱함 속 비범함을 단번에 알아본 자칭 뮤즈 ‘라희’(임윤아)와 함께 설득력 있는 편지쓰기를 위한 맞춤법 수업, 유명세를 얻기 위한 장학퀴즈 테스트, 대통령배 수학경시대회 응시까지! 오로지 기차역을 짓기 위한 ‘준경’만의 노력은 계속되는데...! 포기란 없다 기차가 서는 그날까지!",
                        "인간들의 마을에서 멀리 떨어진 포켓몬들의 낙원 자부숲에서 엄격한 법도를 지키며 무리들과 함께 살아가던 ‘자루도’는 우연히 강가에서 인간의 아이를 발견하게 된다. 차마 아이를 외면할 수 없었던 ‘자루도’는 숲의 법도를 깨고 아이에게 ‘코코’라는 이름을 붙여주며 무리에서 떨어져 둘이서만 살기로 결심한다. 그로부터 10년 뒤, 자신을 포켓몬이라고 믿는 소년 ‘코코’는 자부숲을 찾아온 ‘지우’와 ‘피카츄’를 만나 새로운 세계를 알게 되는데… 한편, 자부숲을 지켜주는 치유의 샘을 노리는 위험한 인간들의 발소리가 들려오기 시작한다! 정글의 아이 ‘코코’는 친구들과 힘을 모아 숲을 지킬 수 있을까?"],
            'running_time':[117, 99], 'poster':['https://movie.naver.com/movie/bi/mi/basic.naver?code=199860#','https://movie.naver.com/movie/bi/mi/basic.naver?code=200894#'], 'genre':['드라마', '애니메이션,모험'], 'rating':['12세 관람가','전체 관람가']}

df = pd.DataFrame(data)

In [6]:
df

,title,release_date,actor,director,summary,running_time,poster,genre,rating
0,기적,2021.09.15,"박정민,이성민,윤아,이수경,김강훈",이장훈,오갈 수 있는 길은 기찻길밖에 없지만 정작 기차역은 없는 마을. 오늘부로 청와대에 ...,117,https://movie.naver.com/movie/bi/mi/basic.nave...,드라마,12세 관람가
1,"극장판 포켓몬스터: 정글의 아이, 코코",2021.09.15,"마츠모토 리카,오오타니 이쿠에,하야시바라 메구미,미키 신이치로,이누야마 이누코",야지마 테츠오,인간들의 마을에서 멀리 떨어진 포켓몬들의 낙원 자부숲에서 엄격한 법도를 지키며 무리...,99,https://movie.naver.com/movie/bi/mi/basic.nave...,"애니메이션,모험",전체 관람가


In [21]:
start_time = time.time()

# 검색 결과 1페이지에서 하나의 영화에 대해서 네이버 영화 리스트와 매칭되는 영화를 찾는 함수

def find_matching_movie(title, director):
    
    # 검색창: input#search_q.input_search
    search_box = driver.find_element_by_css_selector("input#search_q.input_search")
    # 검색버튼: a.btn_search
    search_btn = driver.find_element_by_css_selector("#total_search_form > fieldset > a")
    
#     time.sleep(2)
    
    # 홈페이지에서 제목으로 검색
    search_box.send_keys(title)
    search_btn.click()
    
#     time.sleep(2)

    # 영화 탭 버튼 눌러서 영화에 대한 검색 결과만 보기
    movie_tab_btn = driver.find_element_by_css_selector("#snb_t > li:nth-child(2) > a")
    movie_tab_btn.click()
    
#     time.sleep(2)

    # 일단 해당 영화 제목으로 검색해서 검색 결과가 있는 지 확인
    try:
        # 결과가 하나라도 있는 경우
        # 검색 결과 수 보기
        num_of_movies = driver.find_element_by_css_selector("#content > div.culm2_area > div.culm2_l > h2 > span").text
        num_of_movies = int(num_of_movies.replace('영화', '').replace('(', '').replace(')', '').replace(' ', ''))
        print(f'씨네21에서 {title}에 대해 찾은 검색 결과는 {num_of_movies}건 입니다.')

#         time.sleep(2)
    
        # 영화 검색 결과 페이지에서
        # 일단 1페이지에 내가 찾는 있다고 가정
        # 영화 제목, 감독 일치하면 해당 페이지로 들어가서 정보 가져오기
        
        for i in range(1, 11):
        
            titles = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p.name > a")
            people = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p:nth-child(4) > a:nth-child(2)")
            href = titles[0].get_attribute('href')
        
            time.sleep(2)

            if (title == titles[0].text) and (director == people[0].text):
                return href

            else:
                print('제목과 감독이 일치하지 않습니다.')
        
#         time.sleep(2)
    
    except NoSuchElementException:
        print(f"{title}에 대한 검색 결과가 없습니다.")


# 매칭 되는 경우 테스트: 각각 1페이지에 있는 결과 수는 10, 1개
result = find_matching_movie('기적', '이장훈') # 이건 문제 없이 잘 됨
# result = find_matching_movie('극장판 포켓몬스터: 정글의 아이, 코코', '야지마 테츠오')

# 매칭 안되는 경우 테스트
# result = find_matching_movie('기적', '이기훈')
### 위의 경우에 href에서 list index out of range 뜨는 것 같음
### 왜 titles가 길이가 하나짜리 리스트가 되는건지 잘 모르겟당
### 저 코드 자체가 리스트로 돌려주는 코드인가? movie_tab_btn 보면 그건 아닌거 같은데...
### i로 페이지에 있는 해당 셀렉터 다 돌리니까 그런건가?
# result = find_matching_movie('극장판 포켓몬스터: 정글의 아이, 코코', '야지마')

# url을 리스트로 받아서 다른 함수를 만든다. 그 함수는 url를 차례로 받아서 메타 데이터와 리뷰를 수집한다.

print("--- %s seconds ---" % (time.time() - start_time))

씨네21에서 기적에 대해 찾은 검색 결과는 88건 입니다.
제목과 감독이 일치하지 않습니다.
제목과 감독이 일치하지 않습니다.
제목과 감독이 일치하지 않습니다.
제목과 감독이 일치하지 않습니다.
제목과 감독이 일치하지 않습니다.
--- 18.390122652053833 seconds ---


In [22]:
result # 이제 이 result 변수에 url을 쭉 저장하면 됨 샘플 5개 정도로 저장 잘 되는지 확인해보자
# 모두 잘되는 경우
# 중간에 몇 개 안되는 경우
# 다 안되는 경우
# 안되는 이유도 가지각색으로 만들어서 테스트 해보자

# 매개변수 title과 direct는 받을 때 리스트로 받으면 되나
# map(find_matching_movies, naver_list)
# url 받을 수 있게 함수 수정하고 df에 샘플데이터 가져와서 잘 되는지 다시 돌려보자
# 그리고 다시 테스트 하기 전에 혹시 검색 결과가 0건일 경우에 그 영화 url 못 따왔다고 알 수 있게 따로 저장해야 할 듯

# 이 함수 완성 되면 깃랩에 커밋 해야지

# 이 함수 완성되면 씨네21 작품 상세 페이지 url 매개변수로 받아서 메타 데이터 가져오는 함수랑
# 리뷰 데이터 가져오는 함수 만들어야겠음

'http://www.cine21.com/movie/info/?movie_id=57227'

In [ ]:
아무거나